In [730]:
import pandas as pd
import numpy as np

In [731]:
frames = []
for i in range(4):
    frames.append(pd.read_csv('../data/Sunflower_201'+str(i+2)+'.csv'))

for df in frames:
    df.drop(df.columns[[0,1]],axis=1,inplace=True)

X_ndvi_list = []
Y_list = []

In [732]:
for df in frames:
    X_ndvi_list.append(df.loc[:, df.columns != 'yield'])
    Y_list.append(df.loc[:, df.columns == 'yield'])


In [733]:
for i in range(4):
    X_ndvi_list[i].columns = range(-52,0,1)


In [734]:
frames = []
for i in range(4):
    frames.append(pd.DataFrame.from_csv('../meteo_csv/201'+str(i+2)+'.csv'))


In [735]:
for df in frames:
    df.drop(df.columns[[0, 1,4,6,8,10,12,14,16]], axis=1, inplace=True)
    df['year'] = df['yearmoda'] // 10000
    df['month'] = df['yearmoda'] // 100 % 100
    df['day'] = df['yearmoda'] % 100
    df['sndp'].replace(999.9,0, inplace = True)
    df['yearmoda1'] = df['yearmoda'].astype(str).apply(lambda x: x[:9])
    df.index = pd.to_datetime(df['yearmoda1'], format='%Y%m%d').values
    df.drop(df.columns[[0,16]], axis=1, inplace=True)
    for x in df[df['wdsp'] == 999.9].index:
        df.loc[x,'wdsp'] = np.NaN
    df['wdsp'] = df['wdsp'].interpolate(method='time')
    for x in df[df['mxspd'] == 999.9].index:
        df.loc[x,'mxspd'] = np.NaN
    df['mxspd'] = df['mxspd'].interpolate(method='time')
    for x in df[df['prcp'] == '99.99'].index:
        df.loc[x,'prcp'] = np.NaN
    for x in df.index:
        if (str(df.loc[x,'prcp'])[-1] == 'I') or (str(df.loc[x,'prcp'])[-1] == 'H'):
            df.loc[x,'prcp'] = np.NaN
    for x in df[df['prcp'].notnull()].index:
        df.loc[x,'prcp'] = df.loc[x,'prcp'][:-1]
    for x in df.index:
        if (str(df.loc[x,'max'])[-1] == '*'):
            df.loc[x,'max'] = df.loc[x,'max'][:-1]
        if (str(df.loc[x,'min'])[-1] == '*'):
            df.loc[x,'min'] = df.loc[x,'min'][:-1]
    df['max']=df['max'].astype(np.float64)
    df['min']=df['min'].astype(np.float64)
    df['prcp']=df['prcp'].astype(np.float64)
    df['prcp'] = df['prcp'].interpolate(method='time')
    
    
    


In [736]:
for df in frames:
    df['fog'] = df['frshtt'] // 100000
    df['rain'] = df['frshtt'] // 10000 % 10
    df['snow'] = df['frshtt'] // 1000 % 10
    df['hail'] = df['frshtt'] // 100 % 10
    df['thunder'] = df['frshtt'] // 10 % 10
    df['tornado'] = df['frshtt'] % 10
    df.drop(['frshtt'],axis = 1, inplace = True)


    
    
    

In [737]:
for i in range(3):
    frames[i] = frames[i].fillna(method='bfill')

In [738]:
years = []
years.append(pd.Series(frames[0].values.ravel()[:-40]))
years.append(pd.Series(frames[1].values.ravel()))
years.append(pd.Series(frames[2].values.ravel()))
years.append(pd.Series(frames[3].values.ravel()[:-20]))
for year in years:
    print(year.shape)

(7280,)
(7280,)
(7280,)
(7280,)


In [739]:
years[1]

0         28.40
1         25.00
2       1022.50
3        999.40
4          7.40
5          5.30
6          7.80
7         35.60
8         19.90
9          0.13
10         3.90
11      2013.00
12         1.00
13         1.00
14         1.00
15         0.00
16         0.00
17         0.00
18         0.00
19         0.00
20        32.40
21        30.80
22      1022.80
23       999.90
24         3.30
25         6.80
26         9.70
27        36.70
28        26.20
29         0.13
         ...   
7250       0.00
7251    2013.00
7252      12.00
7253      30.00
7254       1.00
7255       1.00
7256       0.00
7257       0.00
7258       0.00
7259       0.00
7260      32.20
7261      31.70
7262    1030.20
7263    1007.10
7264       1.20
7265       6.80
7266       9.70
7267      34.30
7268      28.80
7269       0.02
7270       0.00
7271    2013.00
7272      12.00
7273      31.00
7274       1.00
7275       0.00
7276       0.00
7277       0.00
7278       0.00
7279       0.00
Length: 7280, dtype: flo

In [740]:
for i in range(4):
    X_ndvi_list[i] = X_ndvi_list[i].transpose()
    print(X_ndvi_list[i].shape)


(52, 26)
(52, 24)
(52, 54)
(52, 44)


In [741]:
for i in range(4):

    df = pd.DataFrame(years[i])
    for j in range(len(Y_list[i].index)-1):
        df = pd.concat([df,years[i]], axis=1)
    df.columns = range(len(Y_list[i].index))
    X1[i] = pd.concat([X_ndvi_list[i],df], join = 'inner')
    X1[i] = X1[i].transpose()
    print(X1[i].shape)

(26, 7332)
(24, 7332)
(54, 7332)
(44, 7332)


In [742]:
X_train, X_test, Y_train, Y_test = pd.concat([X1[0],X1[1],X1[2]]), X1[3], pd.concat([Y_list[0],Y_list[1],Y_list[2]]), Y_list[3] 

In [743]:
X_train.index = range(104)
Y_train.index = range(104)


In [751]:
Y_train = pd.Series(Y_train.values.ravel())

X_tr = X_train.values
Y_tr = Y_train.values
Y_tst = Y_test.values
Y_tr = Y_train.values
X_tr.shape



(104, 7332)

In [721]:
#X_sm_train = X_train.drop(X_train.columns[range(200,-1)], axis=1)

In [752]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(X_tr, Y_tr)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [750]:
X_tr[0]

array([ 0.211,  0.211,  0.211, ...,  0.   ,  0.   ,  0.   ])

In [687]:
X_train.iloc[100,500]

4.2999999999999998

In [724]:
np.isfinite(X_test).sum().sum()

322608